# Obtain weekday prototypes
The main objective of this task is to create every weekday prototypes. We want to model two types of days based on the consumption activity of each building:
- **Active** day.
- **Inactive** day.

Thus, for each counter on the database, we'll get 13 day prototypes (6 working days * 2 types of days + 1 inactive day corresponding to Sundays). Moreover, there are 97 different buildings, so we expect to get 13 * 97 prototypical days.

This previouisly mentioned activity will be defined by calculating the mean of Sundays' consumptions for each building (greater than this value plus some margin will indicate an active day; lower or equal than this value plus the margin should be labelled as an inactive day).

#### Directory structure
./<br></br>
notebook/<br></br>
    &emsp;|--- data-preprocessing<br></br>
    &emsp;&emsp;&emsp;&emsp;|--- weekday_prototypes.ipynb<br></br>
out/<br></br>
    &emsp;|--- raw_consumptions.zip

In [1]:
CONS_PATH = 'C:/Users/thmas/OneDrive - Universidad de Castilla-La Mancha/Informática/TFG/out/'

In [2]:
import pandas as pd

In [3]:
counter_id = 27 # Counter ID example

raw = pd.read_pickle(CONS_PATH + 'raw_consumptions.zip')
raw

,building_id,weekday,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2012-02-24,89,4,NaN,NaN,NaN,NaN,0.000000,25.968207,34.830257,29.201536,...,15.390713,11.000000,13.000000,10.000000,8.000000,8.000000,8.000000,8.000000,8.000000,9.000000
2012-02-25,89,5,8.000000,8.569660,7.830417,7.830404,7.769520,8.895253,7.104747,8.000000,...,9.000000,8.268304,8.837209,9.894487,9.000000,9.617350,8.837233,9.545417,9.000000,8.000000
2012-02-26,89,6,9.000000,9.000000,8.478725,8.521275,9.000000,8.000000,9.000000,8.000000,...,10.000000,9.000000,8.203082,8.837309,8.959610,9.000000,8.552664,9.447336,9.000000,8.064059
2012-02-27,89,0,9.935941,9.000000,10.000000,18.413394,36.837317,35.749289,27.000000,25.762449,...,23.837638,16.837659,16.837655,14.837669,15.863704,14.000000,15.000000,15.000000,15.000000,14.000000
2012-02-28,89,1,15.000000,15.000000,15.000000,23.000000,41.347489,39.652511,34.021674,33.837309,...,24.000000,15.000000,15.000000,14.000000,14.068779,15.931221,15.000000,14.581427,14.418573,15.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-04,2233,2,22.909913,32.403865,33.628554,44.708887,41.996861,41.830621,39.931565,38.410110,...,16.404210,16.274590,10.265586,8.801078,8.523268,8.920408,8.643050,8.642317,8.806212,15.055494
2020-03-05,2233,3,19.531186,37.388927,40.617218,42.142622,44.729416,42.029408,41.356996,37.418092,...,14.259403,15.147316,9.511927,8.137753,8.261290,8.352191,8.169629,14.327638,12.734737,19.528338
2020-03-06,2233,4,24.619904,39.854596,38.970224,46.408685,50.081861,39.844651,40.825715,33.403195,...,22.267127,17.071403,11.609071,8.515376,8.416603,8.297641,8.482840,8.284570,8.371784,8.781833


In [4]:
raw_df = raw[raw['building_id'] == counter_id]
raw_df

,building_id,weekday,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2011-07-26,27,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.846246,22.805419,20.887574,18.846172,18.846057,18.420198,18.000000,18.118729,17.881271,18.000000
2011-07-27,27,2,17.000000,19.000000,18.350795,35.846313,47.846263,50.846379,149.316601,137.956260,...,39.845466,21.500125,19.000000,20.000000,18.000000,18.728452,17.845798,19.425750,18.000000,17.111296
2011-07-28,27,3,18.888704,18.803009,18.845892,35.845713,47.845857,51.845696,162.877204,147.690832,...,42.795578,23.204422,21.000000,22.000000,21.000000,21.023872,19.976128,20.714894,20.285106,20.000000
2011-07-29,27,4,20.000000,21.000000,20.000000,37.788779,45.845705,50.845726,162.822891,143.039176,...,38.845451,21.910895,19.000000,19.000000,19.000000,18.318000,17.845896,17.836104,18.855870,17.846017
2011-07-30,27,5,17.298113,17.000000,17.239697,17.845833,17.914470,18.000000,18.623151,19.376849,...,20.000000,19.235492,19.764508,19.000000,19.000000,19.000000,18.000000,19.000000,18.000000,17.000549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-04,27,2,19.004505,21.021148,38.057053,52.901875,57.864013,71.793254,85.755258,89.709010,...,59.033577,35.644513,25.049160,25.935496,24.935408,24.935482,23.935456,24.935446,23.935382,23.935729
2020-03-05,27,3,23.935578,26.714368,44.533421,61.889281,70.564039,71.070514,71.817131,72.531212,...,59.188564,35.185204,24.552753,24.711419,23.541995,24.350356,24.520554,22.935381,23.935002,23.935854
2020-03-06,27,4,22.935450,24.935351,43.503441,56.356743,68.804180,71.328443,68.749933,73.339526,...,44.501028,28.044172,19.946098,20.946044,19.946038,20.946114,18.946039,20.946113,18.945999,19.946048


### Obtaining prototype measures
In order to get the required measures for every day, first we obtain them for sundays, which are supposed to be inactive days. After that, we base the active/inactive result depending on:

- **Inactive days** &rarr; daily consumption mean within [sundays.mean + 2 * sundays.std, +$\infty$)
- **Active days** &rarr; daily consumption mean within [0, sundays.mean + 2 * sundays.std)

We'll store all this prototypical days (every building has 13, as previously discussed) in a pandas DataFrame for later use

In [5]:
def get_threshold(df: pd.DataFrame) -> float:
    df.loc[:, 'mean'] = df.loc[:, '0':].mean(axis=1) # Calculate daily consumption mean
    mean, std = df['mean'].mean(), df['mean'].std()
    
    del df['mean']
    
    return mean + 2 * std # Calculate threshold based on sundays

In [6]:
def get_prototype(df: pd.DataFrame, counter_id: int, weekday: int, active: bool, type: str = 'mean') -> pd.DataFrame:
    if type == 'std':
        proto = df.loc[:, '0':].std(axis=0) # Calculate hour consumption std
    else:
        proto = df.loc[:, '0':].mean(axis=0) # Calculate hour consumption mean
    
    proto = pd.DataFrame(proto).T # Force it to be a row
    proto.insert(0, 'active', active)
    proto.insert(0, 'weekday', weekday)
    proto.insert(0, 'building_id', counter_id)
    
    return proto

In [7]:
clean_df = raw_df.dropna()

sundays = clean_df[clean_df['weekday'] == 6] # Select Sundays
sundays

,building_id,weekday,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2011-07-31,27,6,18.845599,17.153853,18.000000,18.000000,18.000000,20.000000,19.000000,20.000000,...,19.000000,19.000000,19.000000,19.223473,18.776527,18.000000,18.761067,18.238933,17.452738,17.547262
2011-08-07,27,6,11.949745,11.050255,11.641160,11.358840,11.000000,12.000000,12.024355,12.975645,...,13.000000,13.000000,13.000000,13.000000,11.000000,12.020763,11.979237,12.000000,12.000000,12.000000
2011-08-14,27,6,11.846081,11.959588,12.000000,12.000000,13.000000,13.000000,13.000000,13.000000,...,13.000000,13.000000,13.574492,12.845753,12.579755,12.000000,12.000000,12.000000,12.000000,12.000000
2011-08-21,27,6,11.000000,11.970775,11.029225,11.662400,12.337600,12.000000,13.000000,12.044951,...,12.963477,13.036523,12.000000,13.000000,12.000000,11.192110,11.807890,11.883617,11.116383,12.000000
2011-08-28,27,6,11.000000,11.000000,11.000000,11.000000,11.000000,12.000000,12.000000,13.000000,...,12.829029,12.828929,12.703811,12.954430,11.829000,12.216570,12.000000,12.000000,12.000000,11.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-09,27,6,22.199418,21.935248,21.935288,22.714313,20.945995,21.946031,22.134570,22.756935,...,22.048326,23.843842,22.026832,23.935392,21.935316,21.935441,21.897741,21.946235,20.962361,22.930089
2020-02-16,27,6,23.492198,23.378940,23.935471,22.935478,22.935598,23.935543,23.935378,23.935642,...,24.631899,22.935445,23.935483,23.935471,23.935664,23.935431,22.935426,23.935331,22.935337,23.935503
2020-02-23,27,6,21.946155,21.568840,21.323437,21.946155,21.536587,22.355722,21.946187,22.504338,...,22.452667,21.418263,22.935361,21.935446,21.495630,22.375282,21.935428,21.935402,21.935434,21.935470


In [8]:
threshold = get_threshold(sundays)

mean_proto = get_prototype(sundays, counter_id, 6, False, type='mean') # Get Sundays prototype
mean_proto

D:\PythonEnvironments\pytfg\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
D:\PythonEnvironments\pytfg\lib\site-packages\pandas\core\indexing.py:964: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,building_id,weekday,active,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,27,6,False,21.776376,21.675278,21.671603,21.704343,21.778514,22.113566,22.534185,...,22.800491,22.531329,22.304528,22.206994,22.129812,22.203553,22.167953,22.090445,22.048451,22.019582


In [9]:
std_proto = get_prototype(sundays, counter_id, 6, False, type='std') # Get Sundays prototype
std_proto

,building_id,weekday,active,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,27,6,False,6.03663,6.061523,6.044325,6.040655,6.055625,6.166027,6.236506,...,6.028097,6.011046,5.978995,5.991465,6.013587,6.10415,6.09267,6.156931,6.164345,6.180299


In [10]:
for i in range(0, 6):
    df = clean_df[clean_df['weekday'] == i]
    df.loc[:, 'mean'] = df.loc[:, '0':].mean(axis=1) # Calculate daily consumption mean
    
    df_a = df.loc[df['mean'] > threshold] # Select active days
    mean_proto = mean_proto.append(get_prototype(df_a, counter_id, i, True, type='mean'))
    std_proto = std_proto.append(get_prototype(df_a, counter_id, i, True, type='std'))
    
    df_i = df.loc[df['mean'] <= threshold] # Select inactive days
    mean_proto = mean_proto.append(get_prototype(df_i, counter_id, i, False, type='mean'))
    std_proto = std_proto.append(get_prototype(df_i, counter_id, i, False, type='std'))

del mean_proto['mean']
del std_proto['mean']
mean_proto.reset_index(drop=True, inplace=True), std_proto.reset_index(drop=True, inplace=True)
mean_proto

,building_id,weekday,active,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,27,6,False,21.776376,21.675278,21.671603,21.704343,21.778514,22.113566,22.534185,...,22.800491,22.531329,22.304528,22.206994,22.129812,22.203553,22.167953,22.090445,22.048451,22.019582
1,27,0,True,21.979693,23.543103,31.426908,51.296561,67.658047,83.200861,89.131122,...,51.772829,30.326631,25.144364,24.338024,24.221938,24.013202,23.910511,23.922639,23.912002,23.843911
2,27,0,False,22.238686,22.207350,23.136489,24.923174,26.079460,27.239182,27.492444,...,24.363601,23.134201,22.667096,22.577052,22.586803,22.491049,22.432727,22.396573,22.416018,22.385600
3,27,1,True,23.780270,25.225176,32.939442,52.289284,70.171497,84.275342,89.292622,...,48.428002,30.116703,25.360940,24.645960,24.485554,24.406711,24.403795,24.286735,24.334474,24.195209
4,27,1,False,22.304605,22.406666,22.824097,23.929485,24.724311,25.422168,26.827111,...,23.300737,22.817491,22.632565,22.495167,22.412411,22.486465,22.311501,22.346629,22.276722,22.349848
5,27,2,True,24.292389,25.693246,33.615478,52.904326,68.097874,82.119031,87.616237,...,48.265028,30.064800,25.274034,24.572730,24.343573,24.254407,24.248025,24.159401,24.121965,24.080860
6,27,2,False,22.142884,22.112615,22.747635,23.908835,25.273968,26.295879,27.684022,...,23.253721,22.758464,22.429941,22.333240,22.176426,22.163159,22.175566,22.150295,22.056122,22.103979
7,27,3,True,24.208139,25.566955,33.654202,52.718062,67.634183,80.539656,86.046760,...,47.601667,29.964322,25.240705,24.486387,24.346158,24.218184,24.172398,24.032293,24.021230,23.972843
8,27,3,False,22.059555,22.041019,22.665998,23.511163,24.173073,24.901116,26.573512,...,23.307419,22.838464,22.464422,22.435771,22.293387,22.289189,22.207741,22.252616,22.181020,22.168750
9,27,4,True,24.085880,25.339646,33.525616,51.998909,67.935121,81.325913,86.708406,...,40.849359,25.956118,22.873338,22.550140,22.314342,22.214826,22.156281,22.057118,22.076755,21.969982
